In [1]:
!pip install pyspark

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, sqrt
from time import time

time_start = time()

spark = SparkSession.builder.appName("Read CSV").getOrCreate()

df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("data.csv")

column_names = df.columns[2:][::2]

mean_dict = {}
for column in column_names:
    mean_dict[column] = df.select(avg(column)).first()[0]

correlation_data = []

for i, column1 in enumerate(column_names[:1]):
    for column2 in column_names[i+1:]:
        mean1 = mean_dict[column1]
        mean2 = mean_dict[column2]

        numerator = avg((df[column1] - mean1) * (df[column2] - mean2))
        denominator = sqrt(avg((df[column1] - mean1)**2) * avg((df[column2] - mean2)**2))
        correlation = df.select(numerator / denominator).first()[0]

        correlation_data.append((column1, column2, correlation))

result_df = spark.createDataFrame(correlation_data, ["Device 1", "Device 2", "correlation"]) \
                .orderBy("correlation", ascending=False)

result_df.show()

spark.stop()
print('Time: {}'.format(int(time() - time_start)))


23/05/21 23:43:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/21 23:43:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+-----------+-------------+------------------+
|   Device 1|     Device 2|       correlation|
+-----------+-------------+------------------+
|Count CAT17| Count CVT387|0.7985076940355107|
|Count CAT17|  Count CEK31|0.7968845237687054|
|Count CAT17| Count CJE181|0.7787727867860248|
|Count CAT17| Count CEV011|0.7765231941561195|
|Count CAT17| Count CB1699|0.7269432658044818|
|Count CAT17| Count CLW239|0.6957530123207557|
|Count CAT17|  Count CJM90|0.6932421809590414|
|Count CAT17| Count CEK049|0.6915863523295104|
|Count CAT17| Count CB2105|0.6794805640433301|
|Count CAT17| Count CEE016|0.6661083195572555|
|Count CAT17| Count CB1599|0.6493895044241346|
|Count CAT17|Count CB02411|0.6309373796259062|
|Count CAT17| Count CB1142|0.6054706299442999|
|Count CAT17| Count CB1143|0.5555503531628565|
|Count CAT17| Count COM205|0.5489651275938107|
|Count CAT17|  Count CEK18|0.4468756267881279|
|Count CAT17| Count CB1101|0.1663513465028617|
+-----------+-------------+------------------+

Time: 8
